In [1]:
from pyspark.sql import SparkSession
from urllib.request import urlretrieve
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("ADS project 1")
    .config("spark.sql.repl.eagerEval.enabled", True)  
    .config("spark.executor.memory","4G")
    .config("spark.driver.memory","8G")
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/08/22 01:55:32 WARN Utils: Your hostname, Luo resolves to a loopback address: 127.0.1.1; using 172.17.1.121 instead (on interface eth0)
22/08/22 01:55:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/22 01:55:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.conf.set("spark.sql.parquet.compression.codec", "gzip")

In [3]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
# files that are going to be used in modelling
files = ['../data/curated/tlc_data/2021/final_data/2021--06-final_merged.parquet', \
        '../data/curated/tlc_data/2021/final_data/2021--07-final_merged.parquet', \
        '../data/curated/tlc_data/2021/final_data/2021--08-final_merged.parquet',\
        '../data/curated/tlc_data/2021/final_data/2021--09-final_merged.parquet',\
        '../data/curated/tlc_data/2021/final_data/2021--10-final_merged.parquet',\
        '../data/curated/tlc_data/2021/final_data/2021--11-final_merged.parquet']

In [5]:
# file that is going to be predicted
predict = '../data/curated/tlc_data/2021/final_data/2021--12-final_merged.parquet'

In [6]:
# final change of datatype to meet the requirements of the modelling
def change_type(sdf):
        return sdf.withColumn('is_airport', col('is_airport').cast("integer"))\
                .withColumn('is_rainy', (col('is_rainy') == 'True').cast('integer'))\
                .withColumn('Temperature (F)', col('Temperature (F)').cast('float'))\
                .withColumn('Wind Speed (mph)', col('Wind Speed (mph)').cast('float'))

In [7]:
# encode the categorical features so that they could fit into a numeric model
def one_hot(sdf):
    ohe = OneHotEncoder()
    # attributes that need to be encoded
    ohe.setInputCols(["is_weekend", 'is_airport', 'is_rainy', 'Pickup_time', 'Month'])
    ohe.setOutputCols(["Is_Weekend", "Is_Airport", "Is_Rainy", 'Pickup_Time', 'MONTH'])
    model = ohe.fit(sdf)
    # encode the dataset
    sdf = model.transform(sdf)
    return sdf

In [8]:
path = "../data/curated/tlc_data/"
# read in all files for training
sdf = spark.read.parquet(*files)
sdf = change_type(sdf)
sdf = one_hot(sdf)
# save as the final model data
sdf.write.mode("overwrite").parquet(f'{path}/Model_data.parquet')

In [9]:
# read in the file for testing
sdf = spark.read.parquet(predict)
sdf = change_type(sdf)
sdf = one_hot(sdf)
# save sa the final test data
sdf.write.mode("overwrite").parquet(f'{path}/Test_data.parquet')